# First steps with quantum circuits
This contains first steps with quantum circuits taken out of the coursera course "Practical Quantum computing with IBM Quiskit for Beginners" offered by Packt. The course is based on some older version of Qiskit and I decide for myself to update the content to a recent Qiskit version.

## Installation

In [ ]:
from IPython.display import display, Markdown
import ipywidgets as widgets
import qiskit as q
import qiskit_aer as aer
print(f"Quiskit Version: {q.__version__}")
print(f"Quiskit Aer Version: {aer.__version__}")

Let's create a helper function which displays all the data for the course.

In [ ]:
def ShowSimpleGate(qcircuit, title = '### Quantum Gate', showanimation=False):
    '''
    Calculates  a combination of simple quantum gates and returns an object to display
    Inputs:
        qcircuit:  a qiskit quantum circuit
        showanimation: Boolean. Set to true if you want to see an animated arrow in the bloch sphere (only works for 1qbit systems
    Outputs:
        out: An ipywidget gridspeclayout object you can display
    '''
    ###############################################################
    # Prepare Widget with the grid
    ###############################################################
    # Make a 2x2 output grid and initialize each element with widgets.Output()
    rows=2
    cols=2
    out = widgets.GridspecLayout(rows, cols, layout={
        'width': '60%',
        'align_items': 'flex-start',
        'justify_items': 'flex-start'
        })
    layout = {'width': '100%', 'align_items': 'flex-start', 'justify_items': 'flex-start'}
    [[out.__setitem__((i,j), widgets.Output(layout = layout)) for j in range(cols)] for i in range(rows)]

    ###############################################################
    # Get the text out of the circuit for the description
    ###############################################################
      
    annotation_parts = [title + '\n']
    for gate in qc.data:
        annotation_parts.append('- ' + str(gate.operation.name) + '(')
        for i, bit in enumerate(gate.qubits):
            annotation_parts.append(str(qc.find_bit(bit).index))
            if i==len(gate.qubits)-1:
                annotation_parts.append(')\n')
            else:
                annotation_parts.append(', ')
    annotation = "".join(annotation_parts)

    # top-left box with circuit description and scheme
    out[0,0].append_display_data(Markdown(annotation))
    out[0,0].append_display_data(qc.draw('mpl'))


    ##################################################################
    # Simulation
    #################################################################
    
    # Creating the simulator for the statevector (this is old style  to use the method shown in the course
    # Statevector and unitary matrix actually do not need a simulation run - see example below
    backend = aer.AerSimulator(method='automatic')
    #transpiling is especially important for newer versions of aer (rearranging the circuit to the given platform, here it's only simulation)
    qc_transpiled = q.transpile(qc, backend)
    # add a measurement because we need the state vector later on
    qc_transpiled.save_statevector()

    #Run the circuit
    job = backend.run(qc_transpiled)
    result = job.result()
    statevector = result.get_statevector()

    # put bloch sphere in top-right box
    
    # Animation is time-consuming (thus set to False by default) 
    # there is a deprecation warning, but it still seems to work fine for circuits with 1 qubit
    if showanimation:
        out[0,1].append_display_data(q.visualization.visualize_transition(qc))
    else: 
        # Draw bloch sphere
        out[0,1].append_display_data(q.visualization.plot_bloch_multivector(statevector))
    
    # Histogram in bottom-right box
    counts = result.get_counts()
    out[1,1].append_display_data(q.visualization.plot_histogram(counts))
    
    # put transition matrix and state in bottom-left box
    
    # To get the unitary matrix, you do not need a simlation. quantum_info provides all information
    out[1,0].append_display_data(q.visualization.array_to_latex(q.quantum_info.Operator(qc), prefix='\\text{Unitary matrix} = '))
    out[1,0].append_display_data(q.visualization.array_to_latex(q.quantum_info.Statevector(qc), prefix='\\text{State vector} = '))

    return out
                                 
                     
    

qc = q.QuantumCircuit(1)
qc.x(0)
out = ShowSimpleGate(qc, title = '### A simple circuit - Pauli x-gate')
display(out)

In [ ]:
# Create a quantum circuit with one single qubit
# (the (1) refers to 1 qubit, default spin of the qubit is |0> (up spin)

qc = q.QuantumCircuit(1)

# apply the Pauli x-gate to the first bit (0)
qc.x(0)

# draw the circuit, mpl uses matplotlib to make it a little bit fancier
qc.draw('mpl')

In [ ]:
# Creating the simulator 
backend = aer.AerSimulator(method='statevector')
#transpiling is especially important for newer versions of aer (rearranging the circuit to the given platform, here it's only simulation)
qc_transpiled = q.transpile(qc, backend)
# add a measurement because we need the state vector later on
qc_transpiled.save_statevector()

#Run the circuit
job = backend.run(qc_transpiled)
result = job.result()
statevector = result.get_statevector()

# visualization in the bloch sphere
q.visualization.plot_bloch_multivector(statevector)


#### Other visualization options


In [ ]:
# Although deprecated, this is nice to look at
q.visualization.visualize_transition(qc)

In [ ]:
# Histogram
counts = result.get_counts()
q.visualization.plot_histogram(counts)

#### Initializing circuits

In [ ]:
# Define the initial state as spin down |1>
initial_state = [0, 1] 
qc = q.QuantumCircuit(1)
qc.initialize(initial_state,0)

# apply Pauli x-gate and draw
qc.x(0)
qc.draw('mpl')


### Pauli y-gate



In [ ]:
qc = q.QuantumCircuit(1)
# apply Pauli y-gate and draw
qc.y(0)
qc.draw('mpl')

In [ ]:
# Simulation 
qc_transpiled = q.transpile(qc, backend)
qc_transpiled.save_statevector()
job = backend.run(qc_transpiled)
result = job.result()
statevector = result.get_statevector()
# visualization in the bloch sphere
q.visualization.plot_bloch_multivector(statevector)

The y-gate also moves from |0> to |1>, but it rotates around the y-axis as you can see below. To be very precise, the y-gate moves |0> to i|1>, but the i as a global phase change cannot be measured and is therefore not shown.

In [ ]:
q.visualization.visualize_transition(qc)

## Other 1-qubits shown in the lecture


In [ ]:
qc = q.QuantumCircuit(1)
qc.h(0)
# uncomment the next line to see the applying the gate twice returns to the origin
# qc.h(0)
out = ShowSimpleGate(qc, title = '### Hadamard gate \n This is used to create a superposition')
display(out)

qc = q.QuantumCircuit(1)
qc.y(0)
# uncomment the next line to see the applying the gate twice returns to the origin
# qc.h(0)
out = ShowSimpleGate(qc, title = '### Y-gate \n This is a phase shift gate')
display(out)

qc = q.QuantumCircuit(1)
qc.s(0)
# uncomment the next line to see the applying the gate twice returns to the origin
# qc.h(0)
out = ShowSimpleGate(qc, title = '### S gate \n This creates a phase shift on spin down')
display(out)

